In [67]:
import pandas as pd
import numpy as np

path = ("beegfs/ECAI")
folder = lambda db, eff, we : f"/db{db}_effrate{eff}_we{we}_ECAI"

In [68]:
data = pd.read_csv(path + folder(0, 0.2, 0)*2 + "/t2s.csv").to_numpy()
survival = data.copy()
survival[data > 0] = 1
survival[np.isnan(data)] = 0

print(np.nanmedian(survival, axis=0))
print(np.nanpercentile(survival, 75, axis=0)- np.nanpercentile(survival, 25, axis=0))

[1. 0. 0. 0. 0.]
[0. 0. 0. 0. 0.]


In [69]:
dbs = [0, 1, 10, 100]
effs = [0, 0.2, 0.6, 1]

survivor_database =  {eff: {db : {we : pd.DataFrame() for we in [0, 10]} for db in dbs} for eff in effs}
t2s_database = {eff: {db : {we : pd.DataFrame() for we in [0, 10]} for db in dbs} for eff in effs}

for db in dbs:
    for eff in effs:
        for we in [0, 10]:
            try:
                data = pd.read_csv(path + folder(db, eff, we)*2 + "/t2s.csv").to_numpy()
            except FileNotFoundError:
                continue
            survival = data.copy()
            survival[data > 0] = 1
            survival[np.isnan(data)] = 0
            survivor_database[eff][db][we] = pd.DataFrame(survival, columns=[f"Survival_Ag{i}" for i in range(1, 6)])
            t2s_database[eff][db][we] = pd.DataFrame(data, columns=[f"T2S_Ag{i}" for i in range(1, 6)])


In [114]:
t2s_database[0.2][1][10].describe()

,T2S_Ag1,T2S_Ag2,T2S_Ag3,T2S_Ag4,T2S_Ag5
count,4999.000000,4620.000000,1757.000000,4555.000000,3627.000000
mean,167.996399,373.728788,440.107001,367.517892,409.352633
std,47.118494,60.624478,45.601946,63.640765,54.406576
min,66.000000,195.000000,256.000000,184.000000,222.000000
25%,135.000000,329.000000,412.000000,321.000000,370.000000
50%,160.000000,373.000000,451.000000,365.000000,413.000000
75%,193.000000,418.000000,475.000000,415.000000,453.000000
max,420.000000,500.000000,500.000000,500.000000,500.000000


In [112]:
row = 0.2
grouped_values = np.zeros((4,2))
grouped_values_std = np.zeros((4,2))
grouped_values_median = np.zeros((4,2))
for k, db in enumerate(dbs):
    value = t2s_database[row][db][10].to_numpy()
    grouped_values[k, 0] = np.nanmean(value[:,:int(row*5)])
    grouped_values_std[k, 0] = np.nanstd(value[:,:int(row*5)])
    grouped_values_median[k, 0] = np.nanmedian(value[:,:int(row*5)])
    grouped_values[k, 1] = np.nanmean(value[:,int(row*5)+1:])
    grouped_values_std[k, 1] = np.nanstd(value[:,int(row*5)+1:])
    grouped_values_median[k, 1] = np.nanmedian(value[:,int(row*5)+1:])
    
# print latex row as: grouped_values[0, 0] \pm grouped_values_std[0, 0], grouped_values[0, 1] \pm grouped_values_std[0, 1] & ...

#print(" & ".join([f"{grouped_values[k, 0]:.2f} \\pm {grouped_values_std[k, 0]:.2f}, {grouped_values[k, 1]:.2f} \\pm {grouped_values_std[k, 1]:.2f}" for k in range(4)]))
print(grouped_values_median)

[[149. 448.]
 [160. 401.]
 [147. 383.]
 [151. 393.]]


In [82]:
grouped_values_median

array([[nan, nan],
       [nan, nan],
       [nan, nan],
       [nan, nan]])

In [113]:
row = 0.2
for k, db in enumerate(dbs):
    survivor_np = survivor_database[row][db][10].to_numpy()
    print(f"DB: {db}", survivor_np.mean(axis=0).round(2)*100)
print("--------------")
for k, db in enumerate(dbs):
    survivor_np = survivor_database[row][db][10].to_numpy()
    print(f"DB: {db}", survivor_np.std(axis=0).round(2)*100)
print("--------------")  
for k, db in enumerate(dbs):
    survivor_np = survivor_database[row][db][10].to_numpy()
    print(f"DB: {db}",np.median(survivor_np, axis=0)*100)

DB: 0 [87.  0.  5.  8.  1.]
DB: 1 [100.  92.  35.  91.  73.]
DB: 10 [100.  95.  95.  95.  94.]
DB: 100 [97. 85. 72. 84. 49.]
--------------
DB: 0 [34.  6. 22. 27.  7.]
DB: 1 [ 1. 26. 48. 28. 45.]
DB: 10 [ 0. 22. 22. 23. 24.]
DB: 100 [18. 36. 45. 37. 50.]
--------------
DB: 0 [100.   0.   0.   0.   0.]
DB: 1 [100. 100.   0. 100. 100.]
DB: 10 [100. 100. 100. 100. 100.]
DB: 100 [100. 100. 100. 100.   0.]
